## Utility to publish Power BI reports in a workspace


Defining constants

In [ ]:
TENANT_ID = "your-1111-aaaa-bbbb-tenantid"
KEY_VAULT_SECRET_SCOPE = "key-vault-dbxsecrets"

AZURE_STORAGE_ACCOUNT_NAME = "nextlevelbi01"
AZURE_STORAGE_CONTAINER = "landing-zone"

PBI_WORKSPACE_ID = "1234567-1234-4567-0987-12345678901"
PBI_REPORT_NAME = "Next Level Report.pbix"

### Authenticating in Azure

In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import ClientSecretCredential

client_id = dbutils.secrets.get(scope = KEY_VAULT_SECRET_SCOPE, key = 'next-level-bi-sp-appid')
client_secret = dbutils.secrets.get(scope = KEY_VAULT_SECRET_SCOPE, key = 'next-level-bi-sp-secret')

credential = ClientSecretCredential(TENANT_ID,client_id,client_secret)
datalake_service_client = DataLakeServiceClient(account_url=f"https://{AZURE_STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/",credential=credential)

### Reading report content and publishing to the workspace

In [ ]:
fsc = datalake_service_client.get_file_system_client(AZURE_STORAGE_CONTAINER)
for path in fsc.get_paths():
    if '.pbix' in path.name:
        print(path.name)
        fc = fsc.get_file_client(path.name)
        file_content = fc.download_file()
        publish_report_id = publish_powerbi_report(PBI_WORKSPACE_ID, PBI_REPORT_NAME, file_content)
        print(publish_report_id)